<a href="https://colab.research.google.com/github/xiaodongeast/multimodal/blob/main/smol_VLM_FT_GRPO_step2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q accelerate datasets peft bitsandbytes tensorboard
!pip install -q flash-attn --no-build-isolation
!pip install trl

from google.colab import drive
drive.mount('/content/drive5')

In [ ]:
from google.colab import drive
drive.mount('/content/drive6')

In [ ]:
from huggingface_hub import notebook_login

notebook_login()
#



In [ ]:
#from transformers import Idefics3ForConditionalGeneration,AutoProcessor
#import torch
#from peft import AutoPeftModel
## merge base model and fine-tuned model
#model_id ="HuggingFaceTB/SmolVLM-Instruct"
#merged_model= AutoPeftModel.from_pretrained("/content/drive6/MyDrive/smolvlm-instruct-s4")

#processor = AutoProcessor.from_pretrained(
 #   model_id
#)



In [ ]:
#merged_model = merged_model.merge_and_unload()
#merged_model.save_pretrained("/content/drive6/MyDrive/smolvlm-instruct-s4merged")

In [ ]:
from transformers import Idefics3ForConditionalGeneration,AutoProcessor
import torch
from peft import AutoPeftModel
## merge base model and fine-tuned model
#model_id ="HuggingFaceTB/SmolVLM-Instruct"


#processor = AutoProcessor.from_pretrained(
 #   model_id
#)

#model = Idefics3ForConditionalGeneration.from_pretrained(
#"/content/drive5/MyDrive/smolvlm-instruct-s4merged"
#)

In [ ]:
import os
BASE = "/content/drive5/MyDrive/smolvlm-instruct-s4merged"   # your base model
ADAPTER = "/content/drive5/MyDrive/smolvlm-instruct-r1d"         # <— put your adapter folder here
OUT = "/content/drive5/MyDrive/smolvlm-grpo-merged"
os.makedirs(OUT, exist_ok=True)


In [ ]:
import torch
from transformers import AutoTokenizer, AutoProcessor, AutoModelForCausalLM
from peft import PeftModel
import os

device = "cuda" if torch.cuda.is_available() else ("mps" if torch.backends.mps.is_available() else "cpu")

# Pick a dtype that your device supports well
dtype = torch.float16 if device in ["cuda","mps"] else torch.float32

base = Idefics3ForConditionalGeneration.from_pretrained(
    BASE, torch_dtype=dtype, device_map=None
).to(device)


In [ ]:
# Attach LoRA adapter
model = PeftModel.from_pretrained(base, ADAPTER, is_trainable=False)

# Merge LoRA weights into the base model
merged = model.merge_and_unload(safe_merge=True).to("cpu")

In [ ]:
merged.save_pretrained(OUT, safe_serialization=True)

In [ ]:
merged.push_to_hub("smolvlm-grpo-merged")

In [ ]:
# === GRPO dataset prep for SmolVLM / SmolVLM2 ===
from typing import Dict, Any
from PIL import Image

# Use a consistent instruction; put it in the system message.
instruct ="""Answer question about the image with your reasoning. Follow this format:
<think>
[Your detailed chain-of-thought goes here]
</think>
<answer>
[Your final answer goes here]
</answer>
"""


# Make sure the tokenizer pads on the LEFT (important for GRPO).
# (Some processors already set this; we enforce it to be safe.)
processor.tokenizer.padding_side = "left"

def _normalize_example(example: Dict[str, Any]) -> Dict[str, Any]:
    """Converts input to: question(str), answer_text(str), image(PIL), thought(str or ""), choices(list|None)."""
    img = example["image"]
    if isinstance(img, dict) and "path" in img:
        img = Image.open(img["path"]).convert("RGB")
    elif hasattr(img, "mode"):
        if img.mode != "RGB":
            img = img.convert("RGB")
    else:
        img = Image.open(str(img)).convert("RGB")

    question = str(example.get("question", "")).strip()

    if "multiple_choice_answer" in example and example["multiple_choice_answer"] is not None:
        answer_text = str(example["multiple_choice_answer"]).strip()
    else:
        choices = example.get("choices")
        ans_idx = example.get("answer")
        if isinstance(ans_idx, int) and isinstance(choices, list) and 0 <= ans_idx < len(choices):
            answer_text = str(choices[ans_idx]).strip()
        else:
            answer_text = str(example.get("answer", "")).strip()

    thought = str(example.get("solution", "") or "").strip()
    return {
        "image": img,
        "question": question,
        "answer_text": answer_text,
        "thought": thought,
        "choices": example.get("choices")
    }

def format_grpo_example(raw_ex: Dict[str, Any]) -> Dict[str, Any]:
    """
    Build a GRPO training row:
      - prompt: chat-templated text with <image> token and NO assistant target
      - image: PIL image
      - answer_text: kept for accuracy rewards
      - choices: optional passthrough
    """
    exn = _normalize_example(raw_ex)
    #print(exn)

    messages = [

        {
            "role": "user",
            "content": [
                {"type": "image"},
                {"type": "text", "text":instruct + '\n' + 'Querstion:\n' + exn["question"]}
               # {"type": "text", "text": },
            ],
        },
    ]

    # IMPORTANT: add_generation_prompt=True → the prompt ends where assistant should start generating
    prompt = processor.apply_chat_template(messages, add_generation_prompt=True)

    return {
        "prompt": prompt,
        "image": exn["image"],
        "answer_text": exn["answer_text"],
        #"choices": exn["choices"],  # optional passthrough
        # You can keep other metadata if you like (ids, source, etc.)
    }



In [ ]:
#train_ds = split_ds["train"]
from datasets import load_dataset
from datasets import concatenate_datasets, DatasetDict
from huggingface_hub import notebook_login
ds = load_dataset(("bugkiller2025/vqa_reasoning"))
train_ds = ds['train']
eval_ds = ds['eval']
test_ds = ds['test']
train_ds = train_ds.map(format_grpo_example)
train_ds = train_ds.remove_columns([  "question", "answer", "choices", "question_type","answer_type","image_organ" ,"solution",'phrase_type'])
print(train_ds)
print(train_ds[0]['prompt'])
train_ds[0]['image']
train_ds[0]['answer_text']



In [ ]:
# make sure model and data are correct
import matplotlib.pyplot as plt
example =  train_ds[12]
print(example)
plt.imshow(example['image'])
model.to('cuda')

model_inputs= processor(text=[example['prompt']], images=[example['image']], return_tensors="pt", padding=True).to('cuda')
# Generate predictions
with torch.no_grad():
  outputs = model.generate(**model_inputs,max_new_tokens =200)
trimmed_generated_ids = [out_ids[len(in_ids) :] for in_ids, out_ids in zip( model_inputs.input_ids, outputs)]


# Decode the prediction
prediction = processor.batch_decode(  trimmed_generated_ids, skip_special_tokens=True)
print(prediction)

In [ ]:
import re

import string
from typing import List, Union

def _format_reward(text):

    # now if complete match
    pattern_strict = r'^\s*<think>.*?</think>\s*?\n*\s*?<answer>.*?</answer>\s*$'

    match_strict = re.match(pattern_strict, text, re.DOTALL | re.MULTILINE)
    if match_strict:
      return 1
    else:
      return 0

print(_format_reward('<think> something </think>. '))
input2 = '  <answer>yes</answer>'
input3 =' www <think>gkkkk</think> \n <answer>ssss</answer>'
input4 =' <think>gkkkk</think> \n <answer>ssss</answer> '
input5 =' <think>\nThe enhanced muscles look normal in comparison to other structures of the brain.\n</think>\n<answer>\nNo.'
print(_format_reward(input2))
print(_format_reward(input3))
print(_format_reward(input4))
print(_format_reward(input5))




def format_reward(completions, **kwargs):
    """format: Reward function that checks if the completion has a specific format."""
    completion_contents = [completion for completion in completions]
    rewards = [_format_reward(text) for text in completion_contents]
    #print('format rewards',rewards)
    return rewards


# --- helper: extract only the <answer>...</answer> block if present ---
def extract_xml_answer(text: str) -> str:
    if ('<answer>' not in text) or ('</answer>' not in text):
        return ''
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    answer = _normalize_label(answer.strip())
    return answer

def extract_xml_think(text: str) -> str:
    if ('<think>' not in text) or ('</think>' not in text):
        return ''
    think = text.split("<think>")[-1]
    think = think.split("</think>")[0]
    return think.strip()


# --- helper: plain-text normalization for the fallback path (yes/no, categories) ---


# Optional: small synonym map for yes/no style answers
_YN = {"yes": {"yes", "y", "true", "correct"}, "no": {"no", "n", "false", "incorrect"}}
def _normalize_label(s: str) -> str:
    if s.lower() in _YN["yes"]:
        return "yes"
    elif s.lower() in _YN["no"]:
        return "no"
    else:
        return s


# - reward function
def _count_xml(text) -> float:
    count = 0.0
    if text.count("<think>") == 1:
        count += 0.1 # think use 0.25 for all we try to increase for thnk
    if text.count("</think>") == 1:
        count += 0.1
    if text.count("<answer>") == 1:
        count += 0.1
    if text.count("</answer>") == 1:
        count += 0.1
    return count

def _think_reward(text) ->float: # 570+/- 180 this is perfered length
    # we linearly increase to encourage the length
    if 0 < len(text) < 400:
        return  1.0* len(text) / 400.0 # if 40chr reward 0.1  400char 1
    if  400< len(text) <750 : # previous cut for 750
        return 1.0
    if len(text) >750:
        return 0.25 # bonus for think something
    else:
      return 0.0

print(_think_reward(input5))
print(_think_reward(input3))

def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    try:
      completion_contents = [completion for completion in completions]
    except Exception as e:
      completion_contents = [completion[0]['content'] for completion in completions]
    rewards = [_count_xml(c) for c in completion_contents ]


    return rewards


def think_length_reward_func(completions, **kwargs)  -> list[float]:
    # get think
    try:
      completion_contents = [completion for completion in completions]
    except Exception as e:
      completion_contents = [completion[0]['content'] for completion in completions]
    extracted_think = [extract_xml_think(r) for r in completion_contents ]
    rewards = [_think_reward(r) for r in extracted_think ]

    return rewards


def accuracy_reward( completions, **kwargs) -> list[float]:
    answer_text =  kwargs['answer_text']
    completion_contents = [completion for completion in completions]
    extracted_responses = [extract_xml_answer(r) for r in completion_contents ]
    #print('-'*20, f"Question:\n{q}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}")
    reward = [1.0 if r == a.strip() else 0.0 for r, a in zip(extracted_responses, answer_text)]

    return reward


In [ ]:

from trl import GRPOConfig, GRPOTrainer
from peft import LoraConfig, get_peft_model
from trl import GRPOConfig


# merge


# Configure training arguments using GRPOConfig
training_args = GRPOConfig(
    output_dir="/content/drive5/MyDrive/smolvlm-r1d",
    learning_rate=1e-5,
    remove_unused_columns=False,  # to access the solution column in accuracy_reward
    num_train_epochs=1,
    bf16=True,
    # Parameters that control the data preprocessing
    per_device_train_batch_size= 8,
    max_completion_length=1024,  # default: 256
    num_generations= 8,  # default: 8
    max_prompt_length=None,
    # Parameters related to reporting and saving
    report_to=["tensorboard"],
    logging_steps=10,

    save_strategy="steps",
    save_steps=10,
)



lora_config = LoraConfig(
    task_type="CAUSAL_LM",
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["q_proj", "v_proj"],
)



model = get_peft_model( model, lora_config)

model.print_trainable_parameters()





In [ ]:

trainer = GRPOTrainer(
    model=model,
    processing_class=processor,
    reward_funcs=[accuracy_reward,format_reward,  think_length_reward_func ],
    args=training_args,
    train_dataset=train_ds,
)

In [ ]:
trainer.train()
trainer.save_model("/content/drive5/MyDrive/smolvlm-instruct-r1d")
trainer.push_to_hub("smolvlm-instruct-r1d")

In [ ]:
trainer.save_model("/content/drive5/MyDrive/smolvlm-instruct-r1e")


In [ ]:
from transformers import Idefics3ForConditionalGeneration,AutoProcessor
import torch
from peft import AutoPeftModel
model = Idefics3ForConditionalGeneration.from_pretrained(
"/content/drive5/MyDrive/smolvlm-instruct-r1e"
)